This notebook is only for reference, don't make changes in it. Copy it to your drive (e.g. in ybshmmlchk) and use there.

In [ ]:
## FOR INITIALIZATION 
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')   
# if you want to work with git from this notebook
%cd /content/gdrive/My\ Drive/ybshmmlchk/zatoboj
!mkdir -p ~/.ssh
!cp ../id_rsa ~/.ssh
!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts
!ssh -T git@github.com
!source ../git_config_startup.sh
# switch to the module folder
%cd /content/gdrive/My\ Drive/ybshmmlchk/zatoboj/module
# install module ztpt
!pip install -q -q -e .
# import module ztpt
import ztpt
%load_ext autoreload
%autoreload 2
# check your GPU
!nvidia-smi

In [ ]:
!git pull
!git status

In [ ]:
!git add .
!git commit -m 'write comment here'
!git push

This is configuration of your run.

In [7]:
'''
Model name is automatically set to:

{transformer.model}-
{transformer.version}-
{model.model}=
{model.unique_name}-
{model.max_len}-
{model.batch_size}-
{model.lr}-
{model.freeze_layers}.ckpt
'''

config = ztpt.default_config()

config.transformer.model = 'bert'               # default: albert
config.transformer.version = 'base-uncased'     # default: base-v2

config.model.max_len = 256                      # default: 256
config.model.batch_size = 16                    # default: 16
config.model.model = 'SQUADBERT'                # default: 'SQUADBERT'
config.model.lr = 1e-05                         # default: 1e-05
config.model.freeze_layers = 0                  # default: 0
config.model.signature = 'ed-test'              # default: '' (can be: {your_name}-{version})

config.train.max_epochs = 2                     # default: 1
config.train.val_check_interval = 0.2           # default: 0.2
config.train.limit_train_batches = 0.05           # default: 1.
config.train.limit_val_batches = 0.1             # default: 1.

This is you gym, where you can use `ztpt` module to train and test models. Module `ztpt` includes:


*   `preprocess(config)` -- preprocesses and saves dataset from configuration, unless it already exists on drive
*   `load_data(config)` -- return train,val,test datasets. Load existing or runs preprocessing if needed
*   `create_model(config)` -- return model from configuration. If such model already exists, will suggest to change the name. If you want to load existing model use `load_model`
*   `load_model(config)` -- return saved model, ready to continue traning (todo: set weights&biases to continue the plots, not create new ones)
*   `train_model(config)` -- train model and save result. Everything that preceed traning (preprocessing, creating, loading etc) is done automatically.
*   `evaluate(config)` -- evalute model and show dictionary of accuracy results.

In [ ]:
ztpt.preprocess(onfig)

In [ ]:
train_data, val_data, test_data = ztpt.load_data(config)

In [ ]:
model = ztpt.create_model(config)

In [ ]:
model = ztpt.load_model(config, from_list = True)

In [ ]:
ztpt.train_model(config)

In [ ]:
ztpt.evaluate(model, with_min_start=True)